# Validation Playground

**Watch** a [short tutorial video](https://greatexpectations.io/videos/getting_started/integrate_expectations) or **read** [the written tutorial](https://docs.greatexpectations.io/en/latest/tutorials/validate_data.html?utm_source=notebook&utm_medium=validate_data)

#### This notebook assumes that you created at least one expectation suite in your project.
#### Here you will learn how to validate data in a SQL database against an expectation suite.


We'd love it if you **reach out for help on** the [**Great Expectations Slack Channel**](https://greatexpectations.io/slack)

In [1]:
import json
import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.datasource.types import BatchKwargs
import datetime

2021-06-12T14:08:23-0400 - INFO - Great Expectations logging enabled at 20 level by JupyterUX module.


## 1. Get a DataContext
This represents your **project** that you just created using `great_expectations init`.

In [2]:
context = ge.data_context.DataContext()

## 2. Choose an Expectation Suite

List expectation suites that you created in your project

In [3]:
context.list_expectation_suite_names()

['public.conten_flattened.validation']

In [5]:
expectation_suite_name =  "public.conten_flattened.validation"

## 3. Load a batch of data you want to validate

To learn more about `get_batch`, see [this tutorial](https://docs.greatexpectations.io/en/latest/tutorials/validate_data.html?utm_source=notebook&utm_medium=validate_data#load-a-batch-of-data-to-validate)


In [6]:
# list datasources of the type SqlAlchemyDatasource in your project
[datasource['name'] for datasource in context.list_datasources() if datasource['class_name'] == 'SqlAlchemyDatasource']

['sigir_2021']

In [7]:
datasource_name = 'sigir_2021'

In [9]:
# If you would like to validate an entire table or view in your database's default schema:
batch_kwargs = {'table': "content_flattened", 'datasource': datasource_name}

# If you would like to validate an entire table or view from a non-default schema in your database:
# batch_kwargs = {'table': "content_flattened", "schema": "pulic", 'datasource': datasource_name}

# If you would like to validate the result set of a query:
# batch_kwargs = {'query': 'SELECT YOUR_ROWS FROM YOUR_TABLE', 'datasource': datasource_name}



batch = context.get_batch(batch_kwargs, expectation_suite_name)
batch.head()

2021-06-12T14:10:12-0400 - INFO - 	0 expectation(s) included in expectation_suite.


,product_sku_hash,organization_id,ingestion_timestamp_epoch_ms,item_vector,image_vector,price_bucket
0,4567ac170b580026a565edf35120580efc8d14228c5611e011e1d2625a8188f5,4c3773ad-47fb-4209-bb55-a5a1b10e3d4c,1.622856e+09,"[0.3508215844631195,-0.26332706212997437,0.16586415469646454,-0.11209996789693832,-0.04904332756996155,-0.15326720476150513,-0.04274549335241318,-0.20716765522956848,-0.032622337341308594,-0.013592041097581387,0.24418453872203827,0.004179989919066429,0.04901127889752388,-0.05818378925323486,0.024211060255765915,-0.06378769129514694,-0.10051038861274719,-0.05661046504974365,-0.10162103921175003,0.08017785847187042,-0.07490655779838562,-0.02638772875070572,-0.06848417222499847,0.055211566388607025,-0.06801152229309082,-0.0006919335573911667,0.007154329214245081,-0.09328433871269226,0.023818781599402428,-0.010523619130253792,-0.026335446164011955,0.007179875858128071,0.04122491180896759,0.02236172743141651,0.03294681757688522,-0.046759698539972305,-0.03663371503353119,-0.060177966952323914,0.01563987135887146,-0.0012872030492872,-0.04256240278482437,-0.028577135875821114,0.061961546540260315,0.03775552660226822,0.007312982343137264,0.0422518327832222,0.023248394951224327,-0.016795670613646507,0.011034607887268066,-0.005703331902623177]","[411.31151955467567,-78.00889577899753,105.57672314181225,-16.2925362138567,-170.50402211054984,-52.42347205783106,78.14832692005514,-90.12238154320372,56.11708743826246,7.444791400785469,89.40822615771079,-57.7964568187617,9.37450370796168,41.73255990129773,-0.8052898469466179,69.33751334445047,-20.245637632060227,39.33480951943266,26.319551315728788,4.429640269496289,-3.4022179184586943,-61.41335241084148,-40.296444000001216,49.68745954028121,32.9109576914508,-4.969818419342273,46.57702542970442,-78.19838357256926,-45.748793655085414,-64.51598938431745,5.026253595301569,61.443711119778925,-81.38106816590854,18.450467702166137,-10.350375721975169,54.883079203938934,72.70085142976987,-14.944075604399323,-43.15909145381682,31.729304182851415,-10.516483787760505,-90.43265550235941,9.443044147374968,3.8514222626645527,0.21718250536093464,-0.4932529993840461,10.907042640843947,-38.803612696252046,-10.124990206720923,-18.202225914130214]",10.0
1,404837022de2a89318828818e44b768476334dcb0cbff1e7ac7eb02320dad6f3,4c3773ad-47fb-4209-bb55-a5a1b10e3d4c,1.622856e+09,[],[],
2,183f7a48d139c1c9c76a6eecf96f49091ffef971cb99cf2ebe4fa0a45e66e82a,4c3773ad-47fb-4209-bb55-a5a1b10e3d4c,1.622856e+09,[],[],
3,15b83f3097368d49c79d09a245266fdd18c51b8b2b18cc3fdfd9f229f77227bb,4c3773ad-47fb-4209-bb55-a5a1b10e3d4c,1.622856e+09,[],[],
4,b358c101b6f75b7edc9d673d4eba2f054b056ac4de7fb89a553e135cdefb015c,4c3773ad-47fb-4209-bb55-a5a1b10e3d4c,1.622856e+09,[],[],


## 4. Validate the batch with Validation Operators

`Validation Operators` provide a convenient way to bundle the validation of
multiple expectation suites and the actions that should be taken after validation.

When deploying Great Expectations in a **real data pipeline, you will typically discover these needs**:

* validating a group of batches that are logically related
* validating a batch against several expectation suites such as using a tiered pattern like `warning` and `failure`
* doing something with the validation results (e.g., saving them for a later review, sending notifications in case of failures, etc.).

[Read more about Validation Operators in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/validate_data.html?utm_source=notebook&utm_medium=validate_data#save-validation-results)

In [10]:
# This is an example of invoking a validation operator that is configured by default in the great_expectations.yml file

"""
Create a run_id. The run_id must be of type RunIdentifier, with optional run_name and run_time instantiation
arguments (or a dictionary with these keys). The run_name can be any string (this could come from your pipeline
runner, e.g. Airflow run id). The run_time can be either a dateutil parsable string or a datetime object.
Note - any provided datetime will be assumed to be a UTC time. If no instantiation arguments are given, run_name will
be None and run_time will default to the current UTC datetime.
"""

run_id = {
  "run_name": "andrew_test_1",  # insert your own run_name here
  "run_time": datetime.datetime.now(datetime.timezone.utc)
}

results = context.run_validation_operator(
    "action_list_operator",
    assets_to_validate=[batch],
    run_id=run_id)

DataContextError: No validation operator `action_list_operator` was found in your project. Please verify this in your great_expectations.yml

## 5. View the Validation Results in Data Docs

Let's now build and look at your Data Docs. These will now include an **data quality report** built from the `ValidationResults` you just created that helps you communicate about your data with both machines and humans.

[Read more about Data Docs in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/validate_data.html?utm_source=notebook&utm_medium=validate_data#view-the-validation-results-in-data-docs)

In [ ]:
context.open_data_docs()

## Congratulations! You ran Validations!

## Next steps:

### 1. Read about the typical workflow with Great Expectations:

[typical workflow](https://docs.greatexpectations.io/en/latest/getting_started/typical_workflow.html?utm_source=notebook&utm_medium=validate_data#view-the-validation-results-in-data-docs)

### 2. Explore the documentation & community

You are now among the elite data professionals who know how to build robust descriptions of your data and protections for pipelines and machine learning models. Join the [**Great Expectations Slack Channel**](https://greatexpectations.io/slack) to see how others are wielding these superpowers.